In [1]:
pip install huggingface_hub torchvision

In [2]:
!pip install -q ttach

# Установка библиотек и Импорты

In [3]:
# Ячейка 1: Установка и Импорты
!pip install -q kagglehub timm albumentations

import kagglehub
import albumentations as A
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision.models as tv_models
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score
from huggingface_hub import hf_hub_download
import timm
import os
import json
import random
from collections import defaultdict
from glob import glob
from tqdm import tqdm

print("Библиотеки установлены и импортированы.")

Библиотеки установлены и импортированы.


In [4]:
# НОВАЯ ЯЧЕЙКА: Подключение Google Диска
from google.colab import drive
import os

# Подключаем диск (потребуется дать разрешение)
drive.mount('/content/drive')

# Создаем папку на вашем Google Диске для сохранения моделей
SAVE_DIR = '/content/drive/MyDrive/Colab_Models/Fruit_Classification/'
os.makedirs(SAVE_DIR, exist_ok=True)

print(f"Все модели будут надежно сохраняться в: {SAVE_DIR}")

Mounted at /content/drive
Все модели будут надежно сохраняться в: /content/drive/MyDrive/Colab_Models/Fruit_Classification/


# Блок 2: Настройка окружения и Константы

In [5]:
# Ячейка 2: Конфигурация

# Фиксируем Random Seed
SEED = 9999
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Определяем устройство
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

# Словарь классов
class_to_idx = { "Апельсин": 0, "Бананы": 1, "Груши": 2, "Кабачки": 3, "Капуста": 4,
                 "Картофель": 5, "Киви": 6, "Лимон": 7, "Лук": 8, "Мандарины": 9,
                 "Морковь": 10, "Огурцы": 11, "Томаты": 12, "Яблоки зелёные": 13, "Яблоки красные": 14 }

Device: cuda


# Блок 3: Загрузка данных (Kaggle)

In [7]:
# Ячейка 3: Скачивание данных
kagglehub.login() # Может потребовать ввода токена

dl_lab_1_image_classification_path = kagglehub.competition_download('dl-lab-1-image-classification')
print(f'Data source import complete. Path: {dl_lab_1_image_classification_path}')

train_path_full = os.path.join(dl_lab_1_image_classification_path, 'train', 'train')
test_images_dir = os.path.join(dl_lab_1_image_classification_path, 'test_images', 'test_images')
sample_sub_path = os.path.join(dl_lab_1_image_classification_path, 'sample_submission.csv')

100%|██████████| 194M/194M [00:11<00:00, 17.5MB/s]

Extracting files...


Data source import complete. Path: /root/.cache/kagglehub/competitions/dl-lab-1-image-classification
Kaggle credentials set.


# Блок 4: Подготовка Датасета и Аугментации


In [8]:
# Ячейка 4: Классы Датасета и Аугментации
from albumentations.pytorch import ToTensorV2

train_transforms = A.Compose([
    A.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0), p=0.5),

    # ИСПРАВЛЕНИЕ 1: Убрал always_apply (Resize и так применяется всегда)
    A.Resize(height=224, width=224),

    # 2. Геометрия
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),

    # ИСПРАВЛЕНИЕ 2: Заменили устаревший ShiftScaleRotate на современный Affine
    A.Affine(scale=(0.95, 1.05), translate_percent=(-0.05, 0.05), rotate=(-180, 180), p=0.5),

    # 3. Эффекты
    A.OneOf([
        A.GaussianBlur(blur_limit=(3, 7), p=1.0),
        A.MotionBlur(blur_limit=(3, 7), p=1.0),
        # ИСПРАВЛЕНИЕ 3: Вернули правильное название GaussNoise (без "ian")
        A.GaussNoise(p=1.0),
    ], p=0.2),

    # 4. Свет и Цвет
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=15, val_shift_limit=10, p=0.3),

    # 5. Блики и перекрытия
    A.CoarseDropout(num_holes_range=(1, 8), hole_height_range=(1, 16), hole_width_range=(1, 16), fill_value=0, p=0.2),

    # 6. Финал
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

val_transforms = A.Compose([
    # ИСПРАВЛЕНИЕ 1: Убрали always_apply
    A.Resize(height=224, width=224),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# Класс Датасета
class FruitDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image_filepath = row['filepath']
        label = row['label_idx']

        # Читаем
        image = cv2.imdecode(np.fromfile(image_filepath, dtype=np.uint8), cv2.IMREAD_UNCHANGED)
        try:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        except Exception as e:
            # Заглушка для битых фото
            image = np.zeros((224, 224, 3), dtype=np.uint8)

        if self.transform is not None:
            image = self.transform(image=image)['image']

        return image, label

# Функция создания DataFrame
def create_dataframe(root_path):
    data = []
    all_images = glob(os.path.join(root_path, '*/*/*.jpg')) + \
                 glob(os.path.join(root_path, '*/*/*.png')) + \
                 glob(os.path.join(root_path, '*/*/*.jpeg'))

    for img_path in tqdm(all_images, desc="Parsing images"):
        norm_path = os.path.normpath(img_path)
        parts = norm_path.split(os.sep)

        plu_id = parts[-2]
        class_name = parts[-3]

        if class_name in class_to_idx:
            data.append({
                'filepath': img_path,
                'label_name': class_name,
                'label_idx': class_to_idx[class_name],
                'stratify_group': f"{class_name}_{plu_id}"
            })

    return pd.DataFrame(data)

/tmp/ipython-input-3101193192.py:30: UserWarning: Argument(s) 'fill_value' are not valid for transform CoarseDropout
  A.CoarseDropout(num_holes_range=(1, 8), hole_height_range=(1, 16), hole_width_range=(1, 16), fill_value=0, p=0.2),


# Блок 5: Парсинг данных (Создание таблицы)

In [9]:
# Ячейка 5: Создание DataFrame
df = create_dataframe(train_path_full)
print(f"Всего изображений: {len(df)}")
df.head()

Parsing images: 100%|██████████| 9889/9889 [00:00<00:00, 486265.47it/s]

Всего изображений: 9889


,filepath,label_name,label_idx,stratify_group
0,/root/.cache/kagglehub/competitions/dl-lab-1-i...,Кабачки,3,Кабачки_3965
1,/root/.cache/kagglehub/competitions/dl-lab-1-i...,Кабачки,3,Кабачки_3965
2,/root/.cache/kagglehub/competitions/dl-lab-1-i...,Кабачки,3,Кабачки_3965
3,/root/.cache/kagglehub/competitions/dl-lab-1-i...,Кабачки,3,Кабачки_3965
4,/root/.cache/kagglehub/competitions/dl-lab-1-i...,Кабачки,3,Кабачки_3965


# Блок 6: Логика Модели (Model Zoo)

In [10]:
# Ячейка 6: Загрузка Модели
def get_model(device, pretrained=True):
    """
    ИЗМЕНЕНО: Используем более глубокую модель tf_efficientnet_b4_ns
    (Она мощнее и глубже, чем B3 и стандартный ResNet50).
    """
    model_name = 'tf_efficientnet_b4_ns' # Было 'tf_efficientnet_b3_ns'
    print(f"Создаем модель: {model_name}")

    # Увеличиваем drop_path_rate для более глубокой модели (борьба с переобучением)
    dp = 0.3 if pretrained else 0.0

    model = timm.create_model(
        model_name,
        pretrained=pretrained,
        num_classes=15,
        drop_path_rate=dp
    )
    model.to(device)
    return model

# Блок 7: Функции Обучения (Engine)

In [13]:
# Ячейка 7: Функции обучения и валидации

@torch.no_grad()
def evaluate(model, dataloader, loss_fn, device):
    model.eval()
    total_loss = 0.0
    all_targets = []
    all_preds = []

    for X, y in dataloader:
        X, y = X.to(device), y.to(device)
        logits = model(X)
        loss = loss_fn(logits, y)
        total_loss += loss.item() * y.size(0)

        all_preds.extend(logits.argmax(dim=1).cpu().numpy())
        all_targets.extend(y.cpu().numpy())

    avg_loss = total_loss / len(dataloader.dataset)
    f1 = f1_score(all_targets, all_preds, average='macro')
    return f1, avg_loss

# ИЗМЕНЕНО: Добавлен аргумент save_dir
def run_fold(fold, train_df, val_df, device, save_dir, n_epochs=25):
    print(f"\n{'='*10} FOLD {fold} {'='*10}")

    train_ds = FruitDataset(train_df, transform=train_transforms)
    val_ds = FruitDataset(val_df, transform=val_transforms)
    train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=2)

    model = get_model(device, pretrained=True)

    labels = train_df['label_idx'].values
    weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
    class_weights = torch.tensor(weights, dtype=torch.float).to(device)

    loss_fn = nn.CrossEntropyLoss(weight=class_weights, label_smoothing=0.1)
    optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-2)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=n_epochs, eta_min=1e-6)

    best_f1 = 0.0
    best_val_loss = float('inf')

    for epoch in range(1, n_epochs + 1):
        model.train()
        train_loss = 0
        pbar = tqdm(train_loader, desc=f"Ep {epoch}", leave=False)

        for X, y in pbar:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            logits = model(X)
            loss = loss_fn(logits, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            pbar.set_postfix(loss=loss.item())

        val_f1, val_loss = evaluate(model, val_loader, loss_fn, device)
        scheduler.step()

        print(f"Ep {epoch}: Train Loss {train_loss/len(train_loader):.4f} | Val Loss {val_loss:.4f} | Val F1 {val_f1:.4f}")

        # ИЗМЕНЕНО: Теперь сохраняем прямо на Google Диск
        if val_f1 > best_f1:
            best_f1 = val_f1
            save_path = os.path.join(save_dir, f"model_fold_{fold}_best_f1.pth")
            torch.save(model.state_dict(), save_path)
            print(f"--> Saved best F1 model to Google Drive ({best_f1:.4f})")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            save_path = os.path.join(save_dir, f"model_fold_{fold}_best_loss.pth")
            torch.save(model.state_dict(), save_path)
            print(f"--> Saved best Loss model to Google Drive ({best_val_loss:.4f})")

    return best_f1

# Блок 8: Запуск обучения (Main Loop)

In [14]:
# Ячейка 8: Запуск Cross-Validation
N_FOLDS = 5
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

fold_scores = []

try:
    split_gen = skf.split(df, df['stratify_group'])
except:
    split_gen = skf.split(df, df['label_idx'])

for fold, (train_idx, val_idx) in enumerate(split_gen):
    train_fold = df.iloc[train_idx].reset_index(drop=True)
    val_fold = df.iloc[val_idx].reset_index(drop=True)

    # ИЗМЕНЕНО: Передаем SAVE_DIR
    score = run_fold(fold, train_fold, val_fold, device, save_dir=SAVE_DIR, n_epochs=25)
    fold_scores.append(score)

print(f"Average F1 across folds: {np.mean(fold_scores):.4f}")

/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name tf_efficientnet_b4_ns to current tf_efficientnet_b4.ns_jft_in1k.
  model = create_fn(



========== FOLD 0 ==========
Создаем модель: tf_efficientnet_b4_ns


Ep 1: Train Loss 1.6056 | Val Loss 1.1080 | Val F1 0.8386
--> Saved best F1 model to Google Drive (0.8386)
--> Saved best Loss model to Google Drive (1.1080)


Ep 2: Train Loss 1.1507 | Val Loss 1.0047 | Val F1 0.8939
--> Saved best F1 model to Google Drive (0.8939)
--> Saved best Loss model to Google Drive (1.0047)


Ep 3: Train Loss 1.0782 | Val Loss 0.9897 | Val F1 0.8839
--> Saved best Loss model to Google Drive (0.9897)


...

Ep 21: Train Loss 0.7006 | Val Loss 0.8135 | Val F1 0.9525


Ep 22: Train Loss 0.6943 | Val Loss 0.8120 | Val F1 0.9558
--> Saved best Loss model to Google Drive (0.8120)


Ep 23: Train Loss 0.6945 | Val Loss 0.8072 | Val F1 0.9573
--> Saved best F1 model to Google Drive (0.9573)
--> Saved best Loss model to Google Drive (0.8072)


Ep 24: Train Loss 0.6938 | Val Loss 0.8100 | Val F1 0.9576
--> Saved best F1 model to Google Drive (0.9576)


Ep 25: Train Loss 0.6944 | Val Loss 0.8095 | Val F1 0.9575
Average F1 across folds: 0.9527


# Inference (Предсказание)

In [15]:
# Ячейка 9: Создание submission.csv с TTA
import ttach as tta

submission = pd.read_csv(sample_sub_path)
models = []

for fold in range(N_FOLDS):
    model = get_model(device, pretrained=False)

    # ИЗМЕНЕНО: Читаем сохраненную модель с Google Диска
    path = os.path.join(SAVE_DIR, f"model_fold_{fold}_best_loss.pth")

    if os.path.exists(path):
        model.load_state_dict(torch.load(path, map_location=device))
        model.eval()

        tta_model = tta.ClassificationTTAWrapper(model, tta.aliases.d4_transform())
        models.append(tta_model)
        print(f"Модель {fold} успешно загружена с Google Диска.")
    else:
        print(f"ВНИМАНИЕ: Файл {path} не найден!")

predictions = []

with torch.no_grad():
    for img_id in tqdm(submission["image_id"], desc="Inference with TTA"):
        path = os.path.join(test_images_dir, img_id)
        img = cv2.imdecode(np.fromfile(path, dtype=np.uint8), cv2.IMREAD_UNCHANGED)

        if img is None:
            predictions.append(0)
            continue

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_tensor = val_transforms(image=img)["image"].unsqueeze(0).to(device)

        fold_logits = []
        for model in models:
            logits = model(img_tensor)
            fold_logits.append(logits)

        avg_logits = torch.mean(torch.stack(fold_logits), dim=0)
        predictions.append(avg_logits.argmax(dim=1).item())

submission["label"] = predictions
submission.to_csv("submission_improved.csv", index=False)
print("Файл submission_improved.csv готов!")

Создаем модель: tf_efficientnet_b4_ns


/usr/local/lib/python3.12/dist-packages/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name tf_efficientnet_b4_ns to current tf_efficientnet_b4.ns_jft_in1k.
  model = create_fn(


Модель 0 успешно загружена с Google Диска.
Создаем модель: tf_efficientnet_b4_ns
Модель 1 успешно загружена с Google Диска.
Создаем модель: tf_efficientnet_b4_ns
Модель 2 успешно загружена с Google Диска.
Создаем модель: tf_efficientnet_b4_ns
Модель 3 успешно загружена с Google Диска.
Создаем модель: tf_efficientnet_b4_ns
Модель 4 успешно загружена с Google Диска.


Inference with TTA: 100%|██████████| 2503/2503 [30:58<00:00,  1.35it/s]

Файл submission_improved.csv готов!
